# Revenue Leakage and Profitability Analysis in the Hospitality Sector

This notebook analyzes hotel booking data to identify revenue leakage and suggest strategies to improve profitability. We use booking-level data and enrich it using hotel, room, and date dimension tables to create a unified dataset for business insights.


## Step 1: Preparing the Data for Analysis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# FACT TABLE
bookings = pd.read_csv("fact_bookings.csv")

# DIMENSION TABLES
hotels = pd.read_csv("dim_hotels.csv")
rooms = pd.read_csv("dim_rooms.csv")
dates = pd.read_csv("dim_date.csv")


## Load Datasets

We load the fact table and all dimension tables into pandas DataFrames.
- `fact_bookings.csv` – main transactional data
- `dim_hotels.csv` – hotel names, cities, categories
- `dim_rooms.csv` – room type mapping
- `dim_date.csv` – calendar features for time-based analysis

In [3]:
bookings = bookings.merge(hotels, on='property_id', how='left')


In [4]:
bookings = bookings.merge(rooms, left_on='room_category', right_on='room_id', how='left')
bookings.rename(columns={'room_class': 'room_type'}, inplace=True)
bookings.drop(columns=['room_id'], inplace=True)


In [5]:
# Convert to date only
bookings['booking_date'] = pd.to_datetime(bookings['booking_date']).dt.date
dates['date'] = pd.to_datetime(dates['date'], format='%d-%b-%y', errors='coerce').dt.date

# Merge
bookings = bookings.merge(
    dates[['date', 'mmm yy', 'week no', 'day_type']],
    left_on='booking_date',
    right_on='date',
    how='left'
)
bookings.drop(columns=['date'], inplace=True)

# Optional: Filter out unmatched rows
bookings = bookings[bookings['mmm yy'].notnull()]


In [6]:
print(bookings.info())
print(bookings.head())


<class 'pandas.core.frame.DataFrame'>
Index: 128874 entries, 5 to 134589
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   booking_id         128874 non-null  object 
 1   property_id        128874 non-null  int64  
 2   booking_date       128874 non-null  object 
 3   check_in_date      128874 non-null  object 
 4   checkout_date      128874 non-null  object 
 5   no_guests          128874 non-null  int64  
 6   room_category      128874 non-null  object 
 7   booking_platform   128874 non-null  object 
 8   ratings_given      54292 non-null   float64
 9   booking_status     128874 non-null  object 
 10  revenue_generated  128874 non-null  int64  
 11  revenue_realized   128874 non-null  int64  
 12  property_name      128874 non-null  object 
 13  category           128874 non-null  object 
 14  city               128874 non-null  object 
 15  room_type          128874 non-null  object 
 16  mmm yy 

## Data Enrichment: Merging Dimension Tables

To create a fully unified and analysis-ready dataset, we merged the following dimension tables into our primary fact table (`fact_bookings.csv`). Each merge added meaningful context to enhance our insights.

### 1. Merge with `dim_hotels.csv`
- **Join key**: `property_id`
- **Added columns**: `property_name`, `category` (hotel type), `city`
- **Purpose**: Enables location-based and hotel-level analysis (e.g., revenue by city or category)

### 2. Merge with `dim_rooms.csv`
- **Join key**: `room_category` (fact) → `room_id` (dim)
- **Added column**: `room_class` (renamed as `room_type`)
- **Purpose**: Enables analysis of profitability and leakage by room type (Standard, Elite, etc.)

### 3. Merge with `dim_date.csv`
- **Join key**: `booking_date` (fact) → `date` (dim)
- **Date format**: Converted both columns to plain `date` (not datetime) before merging
- **Added columns**: `mmm yy` (month), `week no`, `day_type` (weekend/weekday)
- **Purpose**: Enables time-based trend analysis (e.g., revenue by month or weekend)

After all merges, our dataset has:
- Detailed transaction info
- Hotel & room context
- Temporal features

This enriched dataset allows for deep business insights and supports effective revenue leakage analysis.


In [7]:
print("Duplicate rows:", bookings.duplicated().sum())


Duplicate rows: 0


In [8]:
print("Nulls in each column:")
print(bookings.isnull().sum().sort_values(ascending=False))


Nulls in each column:
ratings_given        74582
property_id              0
booking_id               0
booking_date             0
check_in_date            0
no_guests                0
checkout_date            0
room_category            0
booking_platform         0
booking_status           0
revenue_generated        0
revenue_realized         0
property_name            0
category                 0
city                     0
room_type                0
mmm yy                   0
week no                  0
day_type                 0
dtype: int64


In [9]:
null_percentage = bookings['ratings_given'].isnull().mean() * 100
print(f"Missing ratings: {null_percentage:.2f}%")


Missing ratings: 57.87%


In [10]:
print("Final shape:", bookings.shape)


Final shape: (128874, 19)


## Clean and Validate the Final Dataset

We check for:
- Duplicates
- Null values
- Column consistency
This ensures our dataset is clean and ready for analysis.

